In [1]:
import numpy as np
import pandas as pd

import warnings
warnings.filterwarnings('ignore')
import pickle
from sklearn.preprocessing import PowerTransformer
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from xgboost import XGBRegressor
import torch
import torch.nn as nn

import torch.nn.functional as F

import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split


import numpy as np
from sklearn.metrics import mean_squared_error
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input, LeakyReLU
from tensorflow.keras.initializers import HeNormal
from tensorflow.keras.callbacks import EarlyStopping
from sklearn.model_selection import train_test_split
import numpy as np

import numpy as np
from sklearn.metrics import mean_squared_error

In [2]:
X_remove = ['X0', 'X4', 'X5', 'X6', 'X7', 'X9', 'X11', 'X13', 'X15', 'X16', 'X20', 'X22', 'X23', 'X25', 'X28', 'X29', 'X30', 'X34', 'X38', 'X39', 'X41']

In [ ]:
test = pd.read_parquet('test_data.parquet')
test.head(5)

test.drop(columns = ['timestamp'],  inplace = True)
test.head()

In [5]:
call = []
for col in test.columns.tolist() :
    if 'call' in col:
        call.append(col)
    else:
        continue
len(call)

put = []
for col in test.columns.tolist() :
    if 'put' in col:
        put.append(col)
    else:
        continue
len(put)  

k_call = []
for col in call:
    col.split('iv')[1]
    k_call.append(col.split('iv')[1].split('_')[1])
len(k_call)

call_test = test.drop(columns = put)
call_test.shape


all_values = []
modified = call_test[call]
for i in range(len(modified)):
    value = modified.iloc[i].tolist()  # 26 values
    all_values.extend(value)
len(all_values)

base = call_test.drop(columns = call)
base.shape

repeated_indices = np.repeat(base.index.values, 26)
expanded_col = base.loc[repeated_indices].reset_index(drop=True)

expanded_col['k'] = k_call * len(base)

expanded_col.shape

expanded_col['iv'] = all_values

expanded_col.head(27)

expanded_col['k'] = expanded_col['k'].astype(float)

In [6]:
import numpy as np

# Replace None with np.nan
expanded_col['iv'] = expanded_col['iv'].replace({'None': np.nan})

not_nan_rows = expanded_col[expanded_col['iv'].notna()]
not_nan_rows.head(10)
df = not_nan_rows
nan_rows = expanded_col[expanded_col['iv'].isna()]
new_df = nan_rows
new_df.head(10)
print(new_df.shape)
df.shape


(188055, 45)


(125635, 45)

In [7]:
x_call = new_df.drop(columns = ['iv']+X_remove)
x_call = StandardScaler().fit_transform(x_call)

from tensorflow.keras.models import load_model

# Load the trained model from the .h5 file
model_call = load_model('call_mlp_model7.h5', compile = False)

# Make predictions
y_pred = model_call.predict(x_call)

# Optional: flatten if single output
y_pred = y_pred.flatten()





new_df['iv'] = y_pred

5877/5877 ━━━━━━━━━━━━━━━━━━━━ 7s 1ms/step


In [8]:
new_df.head(27)

,underlying,X0,X1,X2,X3,X4,X5,X6,X7,X8,...,X34,X35,X36,X37,X38,X39,X40,X41,k,iv
1,24735.9,-2.377420e+06,-0.066961,-0.064562,-0.068452,23078.869048,117442.708333,-55935.974702,-0.0,-0.001486,...,-173761.718749,-0.009541,-0.017831,0.000264,2.032521e+06,1.000000e-06,-0.077238,-5.362742e+06,24100.0,0.231792
2,24735.9,-2.377420e+06,-0.066961,-0.064562,-0.068452,23078.869048,117442.708333,-55935.974702,-0.0,-0.001486,...,-173761.718749,-0.009541,-0.017831,0.000264,2.032521e+06,1.000000e-06,-0.077238,-5.362742e+06,24200.0,0.222416
3,24735.9,-2.377420e+06,-0.066961,-0.064562,-0.068452,23078.869048,117442.708333,-55935.974702,-0.0,-0.001486,...,-173761.718749,-0.009541,-0.017831,0.000264,2.032521e+06,1.000000e-06,-0.077238,-5.362742e+06,24300.0,0.213003
5,24735.9,-2.377420e+06,-0.066961,-0.064562,-0.068452,23078.869048,117442.708333,-55935.974702,-0.0,-0.001486,...,-173761.718749,-0.009541,-0.017831,0.000264,2.032521e+06,1.000000e-06,-0.077238,-5.362742e+06,24500.0,0.197811
7,24735.9,-2.377420e+06,-0.066961,-0.064562,-0.068452,23078.869048,117442.708333,-55935.974702,-0.0,-0.001486,...,-173761.718749,-0.009541,-0.017831,0.000264,2.032521e+06,1.000000e-06,-0.077238,-5.362742e+06,24700.0,0.190474
10,24735.9,-2.377420e+06,-0.066961,-0.064562,-0.068452,23078.869048,117442.708333,-55935.974702,-0.0,-0.001486,...,-173761.718749,-0.009541,-0.017831,0.000264,2.032521e+06,1.000000e-06,-0.077238,-5.362742e+06,25000.0,0.210026
13,24735.9,-2.377420e+06,-0.066961,-0.064562,-0.068452,23078.869048,117442.708333,-55935.974702,-0.0,-0.001486,...,-173761.718749,-0.009541,-0.017831,0.000264,2.032521e+06,1.000000e-06,-0.077238,-5.362742e+06,25300.0,0.239695
14,24735.9,-2.377420e+06,-0.066961,-0.064562,-0.068452,23078.869048,117442.708333,-55935.974702,-0.0,-0.001486,...,-173761.718749,-0.009541,-0.017831,0.000264,2.032521e+06,1.000000e-06,-0.077238,-5.362742e+06,25400.0,0.250140
16,24735.9,-2.377420e+06,-0.066961,-0.064562,-0.068452,23078.869048,117442.708333,-55935.974702,-0.0,-0.001486,...,-173761.718749,-0.009541,-0.017831,0.000264,2.032521e+06,1.000000e-06,-0.077238,-5.362742e+06,25600.0,0.275568
17,24735.9,-2.377420e+06,-0.066961,-0.064562,-0.068452,23078.869048,117442.708333,-55935.974702,-0.0,-0.001486,...,-173761.718749,-0.009541,-0.017831,0.000264,2.032521e+06,1.000000e-06,-0.077238,-5.362742e+06,25700.0,0.292078


In [9]:
test.head(10)

new_df['k'] = new_df['k'].astype('int32')

new_df['k'].dtype

for i in range(len(new_df)):
    value = new_df['iv'].iloc[i]
    k = new_df['k'].iloc[i]
    col = f'call_iv_{k}'
    cols_to_match = [f'X{i}' for i in range(42)]  # Include 'underlying' if needed

    row_i = new_df.iloc[i]
    
    # Compare each row in test to row_i
    mask = (test[cols_to_match] == row_i[cols_to_match]).all(axis=1)
    
    # Get matching index
    matching_index = test.index[mask]

    # insert
    test.loc[matching_index, col] = value

In [10]:
test.isna().sum()

underlying       0
call_iv_24000    0
call_iv_24100    0
call_iv_24200    0
call_iv_24300    0
                ..
X37              0
X38              0
X39              0
X40              0
X41              0
Length: 95, dtype: int64

In [11]:
call_submission = test

In [12]:
call_submission.head()

,underlying,call_iv_24000,call_iv_24100,call_iv_24200,call_iv_24300,call_iv_24400,call_iv_24500,call_iv_24600,call_iv_24700,call_iv_24800,...,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41
0,24735.9,0.280939,0.231792,0.222416,0.213003,0.242149,0.197811,0.232439,0.190474,0.222997,...,0.006587,0.002826,-1.737617e+05,-0.009541,-0.017831,0.000264,2.032521e+06,1.000000e-06,-0.077238,-5.362742e+06
1,24766.7,0.270276,0.246724,0.258893,0.229037,0.220194,0.211350,0.233548,0.202052,0.201285,...,0.005777,0.004588,-3.195086e+05,-0.024106,-0.004696,-0.000158,-4.529075e+06,-1.619022e+06,-0.956928,4.624907e+06
2,24896.9,0.247480,0.251731,0.229870,0.221065,0.214869,0.204580,0.194604,0.186636,0.182048,...,0.000829,-0.034944,1.027525e+06,0.028201,0.032234,0.007687,2.663908e+06,0.000000e+00,-0.116264,-2.669766e+06
3,24898.1,0.241888,0.256558,0.220505,0.238820,0.198602,0.186190,0.212976,0.204362,0.199419,...,-0.009323,-0.022969,-4.720074e+06,-0.001513,0.008704,-0.000206,-7.672936e+06,-1.903406e+06,-2.249208,-1.858254e+07
4,24906.5,0.235328,0.228436,0.222983,0.214126,0.202184,0.193430,0.192603,0.175922,0.171134,...,-0.018674,-0.007588,-4.051681e+05,-0.136267,0.002425,-0.002200,1.334469e+06,2.548789e+05,1.999104,3.024212e+06


In [13]:
test = pd.read_parquet('test_data.parquet')
test.drop(columns = ['timestamp'],  inplace = True)
test.head()


,underlying,call_iv_24000,call_iv_24100,call_iv_24200,call_iv_24300,call_iv_24400,call_iv_24500,call_iv_24600,call_iv_24700,call_iv_24800,...,X32,X33,X34,X35,X36,X37,X38,X39,X40,X41
0,24735.9,0.280939,NaN,NaN,NaN,0.242149,NaN,0.232439,NaN,0.222997,...,0.006587,0.002826,-1.737617e+05,-0.009541,-0.017831,0.000264,2.032521e+06,1.000000e-06,-0.077238,-5.362742e+06
1,24766.7,0.270276,NaN,0.258893,NaN,NaN,NaN,0.233548,NaN,NaN,...,0.005777,0.004588,-3.195086e+05,-0.024106,-0.004696,-0.000158,-4.529075e+06,-1.619022e+06,-0.956928,4.624907e+06
2,24896.9,NaN,0.251731,NaN,NaN,0.214869,0.20458,0.194604,NaN,NaN,...,0.000829,-0.034944,1.027525e+06,0.028201,0.032234,0.007687,2.663908e+06,0.000000e+00,-0.116264,-2.669766e+06
3,24898.1,0.241888,NaN,0.220505,NaN,0.198602,0.18619,NaN,NaN,NaN,...,-0.009323,-0.022969,-4.720074e+06,-0.001513,0.008704,-0.000206,-7.672936e+06,-1.903406e+06,-2.249208,-1.858254e+07
4,24906.5,0.235328,NaN,0.222983,0.214126,NaN,NaN,0.192603,NaN,NaN,...,-0.018674,-0.007588,-4.051681e+05,-0.136267,0.002425,-0.002200,1.334469e+06,2.548789e+05,1.999104,3.024212e+06


In [14]:
put_test = test.drop(columns = call)
put_test.shape


all_values_put = []
modified = put_test[put]
for i in range(len(modified)):
    value = modified.iloc[i].tolist()  # 26 values
    all_values_put.extend(value)
len(all_values_put)

base_put = put_test.drop(columns = put)
base_put.shape

repeated_indices = np.repeat(base_put.index.values, 26)
expanded_col_put = base_put.loc[repeated_indices].reset_index(drop=True)

k_put = []
for col in put:
    col.split('iv')[1]
    k_put.append(col.split('iv')[1].split('_')[1])
len(k_put)

expanded_col_put['k'] = k_put * len(base)

expanded_col_put.shape

expanded_col_put['iv'] = all_values_put

expanded_col_put['k'] = expanded_col_put['k'].astype(float)

In [15]:
import numpy as np

# Replace None with np.nan
expanded_col_put['iv'] = expanded_col_put['iv'].replace({'None': np.nan})

not_nan_rows = expanded_col_put[expanded_col_put['iv'].notna()]
not_nan_rows.head(10)
df = not_nan_rows
nan_rows = expanded_col_put[expanded_col_put['iv'].isna()]
new_df = nan_rows
new_df.head(10)
print(new_df.shape)
df.shape

(188449, 45)


(125241, 45)

In [16]:
x_put = new_df.drop(columns = ['iv']+X_remove)

x_put = StandardScaler().fit_transform(x_put)

from tensorflow.keras.models import load_model

# Load the trained model from the .h5 file
model_put = load_model('put_mlp_model4.h5', compile = False)

# Make predictions
put_y_pred = model_put.predict(x_put)

# Optional: flatten if single output
y_pred = put_y_pred.flatten()


5890/5890 ━━━━━━━━━━━━━━━━━━━━ 9s 1ms/step


In [17]:
new_df['iv'] = y_pred

In [18]:
new_df.head()

,underlying,X0,X1,X2,X3,X4,X5,X6,X7,X8,...,X34,X35,X36,X37,X38,X39,X40,X41,k,iv
0,24735.9,-2.377420e+06,-0.066961,-0.064562,-0.068452,23078.869048,117442.708333,-55935.974702,-0.0,-0.001486,...,-173761.718749,-0.009541,-0.017831,0.000264,2.032521e+06,0.000001,-0.077238,-5.362742e+06,23000.0,0.417071
1,24735.9,-2.377420e+06,-0.066961,-0.064562,-0.068452,23078.869048,117442.708333,-55935.974702,-0.0,-0.001486,...,-173761.718749,-0.009541,-0.017831,0.000264,2.032521e+06,0.000001,-0.077238,-5.362742e+06,23100.0,0.396906
2,24735.9,-2.377420e+06,-0.066961,-0.064562,-0.068452,23078.869048,117442.708333,-55935.974702,-0.0,-0.001486,...,-173761.718749,-0.009541,-0.017831,0.000264,2.032521e+06,0.000001,-0.077238,-5.362742e+06,23200.0,0.377364
3,24735.9,-2.377420e+06,-0.066961,-0.064562,-0.068452,23078.869048,117442.708333,-55935.974702,-0.0,-0.001486,...,-173761.718749,-0.009541,-0.017831,0.000264,2.032521e+06,0.000001,-0.077238,-5.362742e+06,23300.0,0.358425
5,24735.9,-2.377420e+06,-0.066961,-0.064562,-0.068452,23078.869048,117442.708333,-55935.974702,-0.0,-0.001486,...,-173761.718749,-0.009541,-0.017831,0.000264,2.032521e+06,0.000001,-0.077238,-5.362742e+06,23500.0,0.320759


In [19]:
call_submission.head()


new_df['k'] = new_df['k'].astype('int32')

new_df['k'].dtype

for i in range(len(new_df)):
    value = new_df['iv'].iloc[i]
    k = new_df['k'].iloc[i]
    col = f'put_iv_{k}'
    cols_to_match = [f'X{i}' for i in range(42)]  # Include 'underlying' if needed

    row_i = new_df.iloc[i]
    
    # Compare each row in test to row_i
    mask = (call_submission[cols_to_match] == row_i[cols_to_match]).all(axis=1)
    
    # Get matching index
    matching_index = call_submission.index[mask]

    # insert
    call_submission.loc[matching_index, col] = value


In [20]:
call_submission.drop(columns = ['underlying']+[f'X{i}' for i in range(42)]).to_csv('submission7.csv',index = False)